In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from quantum_tunneling.workflows import run_bound_states, run_tdse
from quantum_tunneling.viz import create_wavefunction_animation
from quantum_tunneling.fields import apply_field

In [ ]:
potentials = {
    "cusp": {"kind": "cusp", "params": {"V0": 1.0, "alpha": 0.5}},
    "exp_well": {"kind": "exp_well", "params": {"V0": 5.0, "a": 1.0}},
    "soft_barrier": {"kind": "soft_barrier", "params": {"V0": 5.0}},
    "rough": {"kind": "rough", "params": {"V0": 1.0, "k0": 0.6, "levels": 4, "decay": 0.6}},
}
pot_key = "cusp"
state_index = 0
cfg = {
    "potential": potentials[pot_key],
    "grid": {"L": 12.0, "N": 1400},
    "solver": {"k": 6, "hbar": 1.0, "m": 1.0},
    "tdse": {
        "dt": 0.002,
        "duration": 4.0,
        "record_interval": 50,
        "hbar": 1.0,
        "m": 1.0,
        "F": 0.12,  # set to 0.0 for field-free decay
        "cap": {"x_start": 8.0, "x_end": 11.5, "strength": 3.0, "power": 2},
    },
}
cfg

In [ ]:
res = run_bound_states(cfg)
td = run_tdse(cfg, res, state_index=state_index)
frames = td['frames']
survival = td['survival']
flux = td['flux']
times = [f['t'] for f in frames]
E = res['E'][state_index]
E

In [ ]:
plt.figure()
plt.plot(times, survival, label='Survival probability')
plt.xlabel('t')
plt.ylabel('Survival')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()
plt.figure()
plt.plot(times, [f[0] for f in flux], label='Left flux')
plt.plot(times, [f[1] for f in flux], label='Right flux')
plt.xlabel('t')
plt.ylabel('Boundary flux')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

In [ ]:
# Optional: build animation of probability density.
anim = create_wavefunction_animation(
    frames,
    config={"mode": "probability density"},
    x=res['x'],
    Vx=apply_field(res['Vx'], res['x'], cfg['tdse']['F']),
    markers=td.get('cap_markers'),
    psi_scale=1.0,
    psi_offset=0.0,
    interval=50,
    notebook=True,
    generate_progress=False,
)
anim

## Notes
- Fit an exponential to `survival` vs time to estimate escape rate Γ.
- Increase `duration` or reduce `dt` for accuracy; ensure CAP starts outside the initial localization region.
- Swap `pot_key` and `F` to probe how exotic shapes tilt and affect escape.